# 3 layer LSTM Encoder Decoder, dropout. 120 day input, 10 day forecast

In [9]:
import numpy as np
import pandas as pd
import json
import os
import math
import datetime as dt
import matplotlib.pyplot as plt
from numpy import newaxis
from keras.optimizers import Adam
from keras.layers import Conv1D, Dense, Activation, Dropout, LSTM, CuDNNLSTM, BatchNormalization, RepeatVector, TimeDistributed
from keras.models import Sequential, load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras import regularizers

from sklearn.preprocessing import MinMaxScaler

import utils
import model_funcs

%load_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = [16, 9]
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
df = pd.read_csv('data_25_normalized.csv', index_col='Date')
num_skew_end = 15

data = df.values[:-num_skew_end]

In [77]:
df_orig = pd.read_csv('data_25_orig.csv', index_col='Date')
scaler = MinMaxScaler() 
scaler.fit(df_orig[['Price']])
new = df_orig[['Price']][:-num_skew_end]

In [88]:
import os
df_orig = pd.read_csv('data_25_orig.csv', index_col='Date')
scaler = MinMaxScaler() 
scaler.fit(df_orig[['Price']])
new = df_orig[['Price']][:-num_skew_end]
for i in range(20):
    for file in os.listdir('results/' + str(i)):
        if file.endswith(".h5"):
            model = load_model('results/' + str(i) + "/" + file)
            print(model.layers[0].input_shape)
            
            num_input = model.layers[0].input_shape[1]
            num_out = 10
            train_X, train_y, dev_X, dev_y, test_X, test_y, train_y_acc, dev_y_acc, test_y_acc = utils.prepare_data(data, num_input, num_preds=num_out)

            train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
            dev_y = dev_y.reshape((dev_y.shape[0], dev_y.shape[1], 1))
            test_y = test_y.reshape((test_y.shape[0], test_y.shape[1], 1))
            train_y_acc = train_y_acc.reshape((train_y.shape[0], train_y.shape[1], 1))
            dev_y_acc = dev_y_acc.reshape((dev_y.shape[0], dev_y.shape[1], 1))
            test_y_acc = test_y_acc.reshape((test_y.shape[0], test_y.shape[1], 1))

            preds_train = model.predict(train_X).squeeze()
            preds_dev = model.predict(dev_X).squeeze()
            preds_test = model.predict(test_X).squeeze()
    
            train_scaled = scaler.inverse_transform(preds_train[::num_out].reshape(-1,1))
            dev_scaled = scaler.inverse_transform(preds_dev[::num_out].reshape(-1,1))
            test_scaled = scaler.inverse_transform(preds_test[::num_out].reshape(-1,1))

            total = np.zeros((len(new), 1))
            total[-test_scaled.shape[0]:] = test_scaled
            total[-dev_scaled.shape[0]-test_scaled.shape[0]:-test_scaled.shape[0]] = dev_scaled
            total[-train_scaled.shape[0]-dev_scaled.shape[0]-test_scaled.shape[0]:-dev_scaled.shape[0]-test_scaled.shape[0]] = train_scaled
            new['lstm' + str(i)] = total


(None, 110, 25)
(None, 130, 25)
(None, 130, 25)
(None, 100, 25)
(None, 130, 25)
(None, 110, 25)
(None, 100, 25)
(None, 60, 25)
(None, 110, 25)
(None, 70, 25)
(None, 70, 25)
(None, 100, 25)
(None, 90, 25)
(None, 110, 25)
(None, 100, 25)
(None, 120, 25)
(None, 110, 25)
(None, 90, 25)
(None, 130, 25)
(None, 110, 25)


In [90]:
new.tail(35)

,Price,lstm0,lstm1,lstm2,lstm3,lstm4,lstm5,lstm6,lstm7,lstm8,...,lstm10,lstm11,lstm12,lstm13,lstm14,lstm15,lstm16,lstm17,lstm18,lstm19
Date,,,,,,,,,,,,,,,,,,,,,
2019-02-24,3755.2,3737.801270,3810.346924,3732.047119,3787.447998,4086.507568,3850.525391,3750.826660,3678.152832,3817.873047,...,3791.607422,3691.305664,3555.054443,3609.837891,3512.029541,3859.446045,3678.356689,3713.115234,3888.879395,3649.842285
2019-02-25,3833.7,3821.248291,3893.973145,3793.294922,3848.522461,4179.357910,3929.687744,3791.485840,3737.526367,3867.825928,...,3866.698486,3712.045410,3620.128418,3648.802002,3593.785889,3932.474609,3739.843262,3781.516357,3900.496582,3732.405762
2019-02-26,3810.3,3897.528564,3936.967285,3823.056152,3893.297363,4224.092285,3992.412109,3803.594238,3829.475586,3907.757080,...,3988.178223,3702.528809,3682.040771,3659.110352,3650.074219,4011.952393,3785.345459,3845.786621,3896.268799,3810.448486
2019-02-27,3814.6,3931.230469,3934.048096,3858.236084,3896.790527,4230.501465,4022.386475,3816.319824,3922.822510,3944.129395,...,4068.446045,3677.156006,3720.872314,3647.026855,3662.891602,4086.959473,3777.829102,3898.089111,3895.479736,3895.988037
2019-02-28,3816.6,3907.646484,3911.561035,3888.900879,3932.916992,4225.689941,4023.514648,3826.947266,3990.117920,3984.604736,...,4109.537109,3640.561523,3738.631592,3614.986816,3680.354492,4126.190430,3750.243164,3934.854492,3847.458008,3955.814941
2019-03-01,3821.9,3662.373779,3427.908447,3628.896484,3850.396240,3781.592773,3977.952637,3987.453369,3495.103516,4002.352783,...,3550.404541,3539.353760,3825.358398,3852.772705,3438.518799,4084.576172,3622.767822,3815.409912,3679.606445,3896.048096
2019-03-02,3823.1,3653.342773,3333.872803,3593.685791,3859.151611,3811.809570,3796.929199,3985.155762,3469.468018,3956.576660,...,3658.011963,3596.640625,3725.879639,3939.215820,3693.601074,3763.249512,3438.556641,3869.908203,3596.853760,3835.145996
2019-03-03,3809.5,3703.760010,3387.719727,3614.970459,3785.722656,3832.944824,3735.722656,3937.876953,3547.914307,3851.603516,...,3796.240967,3528.771973,3541.871338,3869.064941,3638.434570,3605.736084,3269.937012,3748.352051,3621.246338,3760.953125
2019-03-04,3715.9,3735.904053,3462.993652,3641.229004,3686.862793,3772.805176,3696.565674,3884.802734,3626.887939,3780.837891,...,3856.378662,3450.798340,3390.329346,3776.538086,3515.060791,3497.670898,3232.024170,3708.461670,3650.668945,3800.580811


In [95]:
import keras.backend as K
for i in range(1):
    for file in os.listdir('results/' + str(i)):
        if file.endswith(".h5"):
            model = load_model('results/' + str(i) + "/" + file)
            print(model.optimizer)
            
)

In [115]:
hps = pd.read_csv('results/1/hyperparams.csv')
results_compiled = pd.read_csv('results/1/results.csv')
for i in range(2, 21):
    path = 'results/' + str(i)
    hps=hps.append(pd.read_csv(path + '/hyperparams.csv'))
    results_compiled=results_compiled.append(pd.read_csv(path + '/results.csv'))


In [116]:
hps.to_csv('hps.csv')

In [117]:
results_compiled

,Unnamed: 0,Train MSE,Dev MSE,Test MSE,Train MAE,Dev MAE,Test MAE
0,0,0.000134,0.000264,0.000508,0.007784,0.010382,0.019753
0,0,0.000339,0.000521,0.000598,0.012374,0.014854,0.020893
0,0,0.000097,0.000208,0.000573,0.006852,0.009312,0.017099
0,0,0.000367,0.000564,0.000841,0.012668,0.015125,0.023673
0,0,0.000081,0.000314,0.000163,0.006473,0.010774,0.009897
0,0,0.000360,0.000427,0.001369,0.012307,0.013123,0.028892
0,0,0.000057,0.000134,0.000241,0.005586,0.007261,0.012466
0,0,0.000081,0.000197,0.001064,0.005975,0.008777,0.026890
0,0,0.000308,0.000355,0.002409,0.011114,0.011227,0.040185
0,0,0.000021,0.000118,0.000467,0.003302,0.006156,0.017604


In [113]:
hps

,Unnamed: 0,batch_size,decay,dropout,lr,num_units,wlen
0,0,16,9.254787e-05,-0.498498,0.000009,100,90
0,0,64,3.409154e-07,-0.292962,0.000013,300,130
0,0,128,2.495400e-06,-0.277330,0.004990,50,100
0,0,64,1.935222e-07,-0.489206,0.000014,250,130
0,0,16,1.792585e-05,-0.439500,0.003331,100,110
0,0,128,5.312649e-06,-0.236561,0.000015,250,100
0,0,64,1.739825e-06,-0.394724,0.001145,100,60
0,0,16,2.896360e-04,-0.451916,0.000095,300,110
0,0,32,1.083132e-07,-0.354835,0.000008,150,70
0,0,16,1.709391e-04,-0.340354,0.000308,250,70


In [91]:
new.to_csv('results.csv')